## **Scrapping Data From Banggood Site**

### **Import library**

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import json
import os
import re
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import ElementNotInteractableException

In [ ]:
categorie_name = "ألعلب الهوايات و الأنسان الآلي"

### **Get Links For Sub-Categories**

In [ ]:
data_dict = []

chrome_options = Options()
chrome_options.add_argument("--disable-notifications")

#---------------------- Path Driver -------------------------------------------
driver = webdriver.Chrome(executable_path=r'C:\Users\khali\Desktop\chromedriver-win64\chromedriver.exe', options=chrome_options)

url = "https://ar.banggood.com/index.php?akmClientCountry=MA&"
driver.get(url)

xpath_popup = '//div[@class="newuser-close iconfont icon-cuowutishi exclick"]'
try:
    wait_popup = WebDriverWait(driver, 10)
    button_popup = wait_popup.until(EC.visibility_of_element_located((By.XPATH, xpath_popup)))
    button_popup.click()
except Exception as e:
    print("")

#----------------------------- Nom de Categorie ---------------------------------------------------------
xpath_category = f'//a[@data-string2="{categorie_name}"]' #
wait_category = WebDriverWait(driver, 10)
button_category = wait_category.until(EC.visibility_of_element_located((By.XPATH, xpath_category)))

# Create an ActionChains object
actions = ActionChains(driver)

# Perform the hover action
actions.move_to_element(button_category).perform()

# Introduce a delay to stay hovered for 5 seconds
time.sleep(5)
page_source = driver.page_source

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

#----------------------- Ajuster l'index du tableau selon le classement de nom de categorie dans la liste des Categories ---------------------------
div_element=soup.find_all('div', {'class': 'cate-cnt'})[0]
sous_cat_dd = div_element.find_all('dd', {'data-spm': '0000000vJ'})

# Iterate through each <dd> tag and find the nested <a> tag
for dd_tag in sous_cat_dd:
    a_tag = dd_tag.find('a')

    if a_tag:
        href = a_tag.get('href')
        text = a_tag.text.strip()

        data_dict.append({"Href": href, "Sous-categorie": text})

# Close the webdriver
driver.quit()

In [ ]:
import json

file_path = 'links_to_sub_categories.json'

with open(file_path, 'w',encoding='utf-8') as json_file:
    json.dump(data_dict, json_file,ensure_ascii=False,indent=4)

print(f"The list has been written to {file_path}")

### **Get Links For Articles**

In [ ]:
html_pages = []

for sous_cat in data_dict:
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,900")
    chrome_options.add_argument("--disable-notifications")
    driver = webdriver.Chrome(executable_path=r'C:\Users\khali\Desktop\chromedriver-win64\chromedriver.exe', options=chrome_options)
    driver.get(sous_cat["Href"])

    # Handle pop-up if it appears
    xpath_popup = '//div[@class="newuser-close iconfont icon-cuowutishi exclick"]'
    try:
        wait = WebDriverWait(driver, 10)
        button_popup = wait.until(EC.visibility_of_element_located((By.XPATH, xpath_popup)))
        button_popup.click()
    except Exception as e:
        print("")

    # Define the ul element locator
    ul_xpath_1 = '//ul[@data-spm="0000001N2"]'

    # Scroll to the end of the page slowly
    scroll_distance = 1000  # Adjust the distance as needed
    for _ in range(10):
        driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
        time.sleep(5)  # Adjust the sleep time as needed

    # Scroll to the end of the page slowly
    scroll_distance_1 = 200  # Adjust the distance as needed
    for _ in range(1):
        driver.execute_script(f"window.scrollBy(0, {-scroll_distance_1});")
        time.sleep(5)  # Adjust the sleep time as needed

    # Gather information from the first page (similar to the original code)
    ul_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ul_xpath_1)))
    all_li = ul_element.find_elements(By.TAG_NAME, 'li')

    # Collect href attributes into a list
    href_list = [li.find_element(By.TAG_NAME, 'a').get_attribute('href') for li in all_li]
    if all_li[0].find_element(By.TAG_NAME, 'a').text=="Go back":
        driver.quit()
        continue
    # Store the href_list in html_pages or process as needed
    html_pages.append({"Sous_Categorie": sous_cat["Sous-categorie"], "Href_List": href_list})

    next_page_button = driver.find_element(By.XPATH, '//i[contains(@class, "iconfont icon-Arrow-right next-page")]')
    while next_page_button:
        try:
            # If found, move to the next page button and click
            next_page_button = driver.find_element(By.XPATH, '//i[contains(@class, "iconfont icon-Arrow-right next-page")]')
            ActionChains(driver).move_to_element(next_page_button).click().perform()
            time.sleep(5)  # Adjust the sleep time as needed
            scroll_distance = 1000  # Adjust the distance as needed

            for _ in range(10):
                driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
                time.sleep(5)

            scroll_distance_1 = 200  # Adjust the distance as needed
            for _ in range(1):
                driver.execute_script(f"window.scrollBy(0, {-scroll_distance_1});")
                time.sleep(5)  # Adjust the sleep time as needed

            # Gather information from the second page (similar to the first page)
            ul_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ul_xpath_1)))
            all_li = ul_element.find_elements(By.TAG_NAME, 'li')

            # Collect href attributes into a list
            href_list = [li.find_element(By.TAG_NAME, 'a').get_attribute('href') for li in all_li]


            # Store the href_list in html_pages or process as needed
            html_pages.append({"Sous_Categorie": sous_cat["Sous-categorie"], "Href_List": href_list})
            next_page_button = driver.find_element(By.XPATH, '//i[contains(@class, "iconfont icon-Arrow-right next-page")]')

        except ElementNotInteractableException:
            break

    driver.quit()

In [ ]:
import json

file_path = 'links_to_articles.json'

with open(file_path, 'w',encoding='utf-8') as json_file:
    json.dump(data_dict, json_file,ensure_ascii=False,indent=4)

print(f"The list has been written to {file_path}")

### **Get Articles infos**

In [ ]:
my_article = []

for i in html_pages:
    s_categories = i["Sous_Categorie"]
    for p,lien in enumerate(i["Href_List"]):
        print(f"Sous_Categorie : {s_categories} , link : {p}")
        try:
            chrome_options = Options()
            chrome_options.add_argument("--window-size=1920,900")
            chrome_options.add_argument("--disable-notifications")
            driver = webdriver.Chrome(executable_path=r'C:\Users\khali\Desktop\chromedriver-win64\chromedriver.exe', options=chrome_options)
            driver.get(lien)

            driver.implicitly_wait(10)
            time.sleep(5)
            scroll_distance = 500
            for _ in range(4):
                driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
                time.sleep(5)


            page_source = driver.page_source


            soup = BeautifulSoup(page_source, 'html.parser')

            og_url = soup.find('meta', {'property': 'og:url'})
            og_url = og_url.get('content', '') if og_url else ''

            og_title = soup.find('meta', {'property': 'og:title'})
            og_title = og_title.get('content', '') if og_title else ''

            og_img = soup.find('meta', {'property': 'og:image'})
            og_img = og_img.get('content', '') if og_img else ''

            prix = soup.find('span', {'class': 'main-price'})
            prix = prix.text.strip().split("$")[1] if prix else ''

            evaluation = soup.find('span', {'class': 'star-num js-star-num'})
            evaluation = evaluation.text.strip() if evaluation else ''

            marque = soup.find('div', {'class': 'reviewer-brand'})
            marque = marque.text.strip().split(":")[1] if marque else ''

            div_with_spans = soup.find('div', {'class': 'product-oa-desc'})
            desc=""

            if div_with_spans:
                desc=div_with_spans.text
            else:
                desc=""

            data_dict = {
                'link': og_url,
                'name': og_title,
                'price': prix,
                'ratings': evaluation,
                'image_link': og_img,
                'brand':marque,
                'description':desc,
                'category':f"{categorie_name}",
                'subcategory':s_categories
            }
            my_article.append(data_dict)

        except Exception as e:
            print(f"Error processing {lien}: {str(e)}")

        finally:
            driver.quit()

In [ ]:
import json

# Specify the file path
file_path = 'data_banggood.json'

# Write the list to a JSON file
with open(file_path, 'w',encoding='utf-8') as json_file:
    json.dump(my_article, json_file,ensure_ascii=False,indent=4)

print(f"The list has been written to {file_path}")